In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
import os


import cv2
import shutil
import numpy as np

from PIL import Image, ImageDraw
from ultralytics import YOLO

from utils.utils import list_file_r, copy_image_to_sub_dir, create_yolo_annotation_from_mask

In [ ]:
# validate model
model = YOLO('runs/segment/augmented/weights/best.pt')
val_results = model.val(data='val.yaml', task='detect')

In [6]:
model = YOLO('runs/segment/augmented_orig+supps_large_640_50epochs/weights/best.pt')
inference_results = model(source='datasets/CF_simulation/real_image/test_gs/', save=True, project='real_image_test' ,name='real_image_test', conf=0.70)


image 1/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\0006.tif: 512x640 6 ink traces, 14.0ms
image 2/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\01_line_Radius5_2_Density2000_2000_Width60_Reflection_0.tif: 640x640 8 ink traces, 17.0ms
image 3/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\1.tif: 640x640 6 ink traces, 17.0ms
image 4/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\2.tif: 640x640 4 ink traces, 17.0ms
image 5/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\2400.tif: 512x640 1 ink trace, 15.0ms
image 6/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\3.tif: 640x640 6 ink traces, 17.0ms
image 7/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\4.tif: 640x640 8 ink traces, 17.0ms
image 8/22 d:\xie\Vision\mcp_yolo\datasets\CF_simulation\real_image\test_gs\hdt5_0010.tif: 512x640 6 ink traces, 15.0ms
image 9/22 d:\xie\Vision\mcp_yolo\da

In [162]:
img = Image.open('datasets/CF_simulation/real_image/test/hdt5_1483.tif').convert('1')
#img.show()
img.save('datasets/CF_simulation/real_image/test_bw/hdt5_1483.tif')

In [3]:
import matplotlib.pyplot as plt
import time
#img = cv2.imread('datasets/CF_simulation/real_image/test/hdt5_0010.tif')
#img = Image.open('real_image_test/real_image_test/hdt5_1482.tif') 
img = Image.open('real_image_test/real_image_test7/01_line_Radius5_2_Density2000_2000_Width60_Reflection_0.tif') 
draw = ImageDraw.Draw(img) 
w, h = img.size
for coos in inference_results[0].masks.xy:
    start = time.time_ns()
    # get LMS estimation
    x, y = coos[:,0:1], coos[:,1:]
    X = np.concatenate((np.ones_like(x), x), axis=-1)
    XT = X.transpose()
    XTX = XT @ X
    theta = np.linalg.inv(XTX) @ (XT @ y)
    theta=theta.reshape(-1)
    intercept, slope = theta[0], theta[1]
    
    # get projection distances
    y_hat = X@theta
    x_hat = (y_hat-intercept)/slope
    projection_2 = np.stack((x_hat, y_hat), axis=1)
    a = np.array([[1], [slope]])
    B = np.concatenate((x, y), axis=-1).transpose()
    B[-1] -= intercept
    P = a@(a.transpose()@B)/(a.transpose()@a)
    B[-1] += intercept
    P[-1] += intercept
    E = B-P
    # normalize
    E[0]/=w
    E[1]/=h
    # calculate distance
    distances = (E**2).sum(axis=0)**.5
    estimated_widths = distances.mean()*2
    #estimated_widths_2 = 
    print('Elapsed time: {:.5e}'.format((time.time_ns()-start)/1e9))

    # draw kernel
    draw_points = [(0, int(intercept)), (w, int(intercept+slope*w))]
    draw.line(draw_points, fill='green', width=10)
    # draw projection
    choice = np.random.choice(range(coos.shape[0]))
    random_coo, projection = B.transpose()[choice], P.transpose()[choice]
    draw_points = [tuple(random_coo), tuple(projection)]
    draw.line(draw_points, fill='blue', width=5)
    print(estimated_widths)

#img.show()
plt.imshow(np.array(img))

NameError: name 'inference_results' is not defined

In [47]:
prediction = np.array(
    [0.053740,
0.053740,
0.053740,
0.056569,
0.054447,
0.053740,
0.054447,
0.055861]
)
gt = np.array(
    [0.061518,
0.060811,
0.061518,
0.053033,
0.053033,
0.053033,
0.053033,
0.053033,]
)
np.abs(((prediction-gt)/gt)).mean()

0.06947559352041428

In [45]:
np.abs(np.array([   -0.08642,   -0.069476,   -0.067315,    0.033356,    0.048161,    0.055445,   -0.035815,   0.0042134])).mean()

0.050025175

In [49]:
# check masks
img = Image.open('real_image_test/real_image_test7/01_line_Radius5_2_Density2000_2000_Width60_Reflection_0.tif')
mask = Image.new("RGB", img.size, "black")  


for coos in inference_results[0].masks.xy:
    polygons = ImageDraw.Draw(mask)
    polygons.polygon(coos, fill='white')
    # calculate the individual width
    temp_mask = Image.new("RGB", img.size, "black")
    temp = ImageDraw.Draw(temp_mask)  
    temp.polygon(coos, fill='white')
    temp_mask = temp_mask.convert('1')
    #temp_mask.show()
    temp_mask = np.array(temp_mask)
    
    scanned_heights=np.array([])
    for col in temp_mask.T:
        col_sum = col.sum()
        if col_sum:
            scanned_heights = np.append(scanned_heights,col.sum())
    #print("{:.6f}".format(np.quantile(scanned_heights, 0.5, method='median_unbiased')/2**.5/temp_mask.shape[-1]))
    #print(scanned_heights.max()/2**.5)
#mask.show()
mask = mask.convert('1')
#mask.show()
mask = np.array(mask)
scanned_heights=np.array([])
for col in mask.T:
    col_sum = col.sum()
    if col_sum:
        scanned_heights = np.append(scanned_heights,col.sum())
print("{:.6f}".format(np.quantile(scanned_heights, 0.5, method='median_unbiased')/2**.5/temp_mask.shape[-1]))
mask_matrix = np.array(mask)

0.265872


In [2]:
import torch
torch.cuda.empty_cache()